In [ ]:
from sympy import symbols, Matrix, rot_axis1, rot_axis2, rot_axis3, latex

def Code(mtx, head):
    ret = head + ':\n'
    for i in range(3):
        text = latex(mtx[i, :3], fold_func_brackets=True, mul_symbol=' * ', mat_delim='')
        res = text.replace('\\begin{matrix}', '').replace('\\end{matrix}', '').replace(' &', ',') \
                  .replace('\\sin {θ_{x}}', 'sx').replace('\\cos {θ_{x}}', 'cx') \
                  .replace('\\sin {θ_{y}}', 'sy').replace('\\cos {θ_{y}}', 'cy') \
                  .replace('\\sin {θ_{z}}', 'sz').replace('\\cos {θ_{z}}', 'cz').replace(', - ', ', -')
        if (res[:2] == '- '):
            res = '-' + res[2:]
        ret += 'row(' + str(i) + ') << ' + res + ';\n'
    return ret

# Variables
tx, ty, tz = symbols('t_x, t_y, t_z')
θx, θy, θz = symbols('θ_x, θ_y, θ_z')
# sympy provides rotation matrices of rotating the axes
# but we need to rotate the objects instead -> nagaive angles
Rx, Ry, Rz = rot_axis1(-θx), rot_axis2(-θy), rot_axis3(-θz)
R = Rx * Ry * Rz
t = Matrix([tx, ty, tz])
p = Matrix([tx, ty, tz, θx, θy, θz])

# Cell p
μpx, μpy, μpz = symbols('μ_{px}, μ_{py}, μ_{pz}')
a, b, c, d, e, f = symbols('a, b, c, d, e, f')
μp = Matrix([μpx, μpy, μpz])
Σp = Matrix([[a, b, c], [b, d, e], [c, e, f]])

# Derivatives of R
dRdθx, dRdθy, dRdθz = Matrix.zeros(3), Matrix.zeros(3), Matrix.zeros(3)
ddRdθxθx, ddRdθxθy, ddRdθxθz = Matrix.zeros(3), Matrix.zeros(3), Matrix.zeros(3)
ddRdθyθy, ddRdθyθz, ddRdθzθz = Matrix.zeros(3), Matrix.zeros(3), Matrix.zeros(3)
for i in range(3):
    for j in range(3):
        dRdθx[i, j] = R[i, j].diff(θx)
        dRdθy[i, j] = R[i, j].diff(θy)
        dRdθz[i, j] = R[i, j].diff(θz)
        ddRdθxθx[i, j] = R[i, j].diff(θx).diff(θx)
        ddRdθxθy[i, j] = R[i, j].diff(θx).diff(θy)
        ddRdθxθz[i, j] = R[i, j].diff(θx).diff(θz)
        ddRdθyθy[i, j] = R[i, j].diff(θy).diff(θy)
        ddRdθyθz[i, j] = R[i, j].diff(θy).diff(θz)
        ddRdθzθz[i, j] = R[i, j].diff(θz).diff(θz)


# First Derivatives of μpq = R * μp + t  - μq
jtx = Matrix([1, 0, 0])  # dμpqdtx
jty = Matrix([0, 1, 0])  # dμpqdty
jtz = Matrix([0, 0, 1])  # dμpqdtz
jθx = dRdθx * μp         # dμpqdθx
jθy = dRdθy * μp         # dμpqdθy
jθz = dRdθz * μp         # dμpqdθz

# Second Derivatives of μpq
# Derivative that contains dtx, dty, or dtz is zero
Hθxθx = ddRdθxθx * μp    # ddμpqdθxdθx
Hθxθy = ddRdθxθy * μp    # ddμpqdθxdθy
Hθxθz = ddRdθxθz * μp    # ddμpqdθxdθz
Hθyθy = ddRdθyθy * μp    # ddμpqdθydθy
Hθyθz = ddRdθyθz * μp    # ddμpqdθydθz
Hθzθz = ddRdθzθz * μp    # ddμpqdθzdθz

# First Derivatives of Σpq = R * Σp * R.T + Σq
# Derivative that contains dtx, dty, or dtz is zero
Zθx = (dRdθx * Σp * R.T) + (dRdθx * Σp * R.T).T  # dΣpqdθx
Zθy = (dRdθy * Σp * R.T) + (dRdθy * Σp * R.T).T  # dΣpqdθy
Zθz = (dRdθz * Σp * R.T) + (dRdθz * Σp * R.T).T  # dΣpqdθz

# Second Derivatives of Σpq
# Derivative that contains dtx, dty, or dtz is zero
Zθxθx = (ddRdθxθx * Σp * R.T + dRdθx * Σp * dRdθx.T) + (ddRdθxθx * Σp * R.T + dRdθx * Σp * dRdθx.T).T  # ddΣpqdθxdθx
Zθxθy = (ddRdθxθy * Σp * R.T + dRdθx * Σp * dRdθy.T) + (ddRdθxθy * Σp * R.T + dRdθx * Σp * dRdθy.T).T  # ddΣpqdθxdθy
Zθxθz = (ddRdθxθz * Σp * R.T + dRdθx * Σp * dRdθz.T) + (ddRdθxθz * Σp * R.T + dRdθx * Σp * dRdθz.T).T  # ddΣpqdθxdθz
Zθyθy = (ddRdθyθy * Σp * R.T + dRdθy * Σp * dRdθy.T) + (ddRdθyθy * Σp * R.T + dRdθy * Σp * dRdθy.T).T  # ddΣpqdθydθy
Zθyθz = (ddRdθyθz * Σp * R.T + dRdθy * Σp * dRdθz.T) + (ddRdθyθz * Σp * R.T + dRdθy * Σp * dRdθz.T).T  # ddΣpqdθydθz
Zθzθz = (ddRdθzθz * Σp * R.T + dRdθz * Σp * dRdθz.T) + (ddRdθzθz * Σp * R.T + dRdθz * Σp * dRdθz.T).T  # ddΣpqdθzdθz